<a href="https://colab.research.google.com/github/ahmadabousalem/sentinel-threat-map/blob/main/risk%20colour%20map3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install feedparser pandas --quiet

import feedparser
import pandas as pd

# Define cities you want to monitor
cities = ["Beirut", "Baghdad", "Damascus", "Gaza"]

# RSS news feeds
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml",
    "https://rss.dw.com/rdf/rss-en-all"
]

# Fetch and filter articles
def fetch_articles():
    articles = []
    for feed_url in rss_feeds:
        feed = feedparser.parse(feed_url)
        for entry in feed.entries:
            for city in cities:
                if city.lower() in entry.title.lower() or city.lower() in entry.summary.lower():
                    articles.append({
                        "City": city,
                        "Title": entry.title,
                        "Summary": entry.summary[:200],
                        "Published": entry.published if "published" in entry else "N/A",
                        "Link": entry.link
                    })
    return pd.DataFrame(articles)

# Fetch and display
df = fetch_articles()
df.head(10)  # Show first 10 articles


,City,Title,Summary,Published,Link
0,Gaza,The secret system Hamas uses to pay government...,Observers are baffled at how Hamas can pay off...,"Wed, 06 Aug 2025 16:48:57 GMT",https://www.bbc.com/news/articles/c1kz42j92jmo...
1,Gaza,"Twenty killed after trucks overturn in Gaza, H...",Four commercial trucks were looted in central ...,"Wed, 06 Aug 2025 20:06:46 GMT",https://www.bbc.com/news/articles/cz60dy2908do...
2,Gaza,Netanyahu to propose full reoccupation of Gaza...,The plan will involve a complete takeover of G...,"Tue, 05 Aug 2025 15:06:52 GMT",https://www.bbc.com/news/articles/cpqv2qjg5vvo...
3,Gaza,"He is a human skeleton, Gaza hostage's brother...",Ilay David says the Hamas video showing his br...,"Mon, 04 Aug 2025 17:05:21 GMT",https://www.bbc.com/news/articles/cewyk4ezeedo...
4,Gaza,Hundreds of Israeli ex-officials appeal to Tru...,It comes amid reports that PM Netanyahu is pus...,"Mon, 04 Aug 2025 13:01:22 GMT",https://www.bbc.com/news/articles/crkznje8nz8o...
5,Gaza,World leaders condemn videos of emaciated Isra...,Hamas said it would allow Red Cross access if ...,"Sun, 03 Aug 2025 21:15:58 GMT",https://www.bbc.com/news/articles/crr2dwn7q40o...
6,Gaza,Aid group says worker killed by Israeli milita...,The Palestine Red Crescent Society accuses Isr...,"Sun, 03 Aug 2025 21:43:27 GMT",https://www.bbc.com/news/articles/cx2x5eyl676o...
7,Gaza,Trump envoy visits controversial Gaza aid dist...,The visit with the IDF follows near daily repo...,"Fri, 01 Aug 2025 18:14:15 GMT",https://www.bbc.com/news/articles/c8x5jnl5eygo...
8,Gaza,France suspends Gaza evacuations after student...,"The Palestinian student, who arrived in France...","Fri, 01 Aug 2025 17:15:39 GMT",https://www.bbc.com/news/articles/c5yl7n42325o...
9,Gaza,Gazan boy first to be treated in UK for war in...,"Majd, 15, was almost killed by an Israeli tank...","Thu, 31 Jul 2025 18:07:39 GMT",https://www.bbc.com/news/articles/cvgn6979n59o...


In [22]:
pip install feedparser pandas folium --quiet


In [23]:
!pip install feedparser folium pandas --quiet

import feedparser
import folium
import pandas as pd
from datetime import datetime


In [24]:
# Define the cities to monitor
cities = {
    "Beirut": {"lat": 33.8938, "lon": 35.5018},
    "Baghdad": {"lat": 33.3152, "lon": 44.3661},
    "Damascus": {"lat": 33.5138, "lon": 36.2765},
    "Gaza": {"lat": 31.5, "lon": 34.47}
}

# Add your open-source RSS feeds
rss_feeds = {
    "Al Jazeera": "https://www.aljazeera.com/xml/rss/all.xml",
    "BBC": "http://feeds.bbci.co.uk/news/world/rss.xml",
    "DW": "https://rss.dw.com/rdf/rss-en-all",
    "CNN": "http://rss.cnn.com/rss/edition_world.rss",
    "Reuters": "http://feeds.reuters.com/Reuters/worldNews"
}

# Keywords to look for
risk_keywords = ['attack', 'explosion', 'war', 'strike', 'missile', 'death', 'protest', 'conflict', 'rocket', 'terror']

# Create an empty list to store data
data = []

# Loop through feeds
for source, url in rss_feeds.items():
    feed = feedparser.parse(url)
    for entry in feed.entries:
        entry_text = (entry.title + " " + entry.get("description", "")).lower()
        for city in cities:
            if city.lower() in entry_text:
                risk_score = sum(1 for kw in risk_keywords if kw in entry_text)
                data.append({
                    "city": city,
                    "source": source,
                    "title": entry.title,
                    "published": entry.get("published", "N/A"),
                    "score": risk_score
                })

# Convert to DataFrame
df = pd.DataFrame(data)
df = df.groupby("city").agg({"score": "sum"}).reset_index()


In [25]:
# Create the base map
m = folium.Map(location=[33.8938, 35.5018], zoom_start=6)

# Add risk circles
for index, row in df.iterrows():
    city = row['city']
    score = row['score']
    location = [cities[city]["lat"], cities[city]["lon"]]

    folium.CircleMarker(
        location=location,
        radius=5 + score * 2,
        color='red' if score > 3 else 'orange',
        fill=True,
        fill_color='red' if score > 3 else 'orange',
        fill_opacity=0.7,
        popup=f"{city} - Risk Score: {score}"
    ).add_to(m)

# Save and show map
m.save("sentinel_risk_map.html")
m


In [26]:
import feedparser
import pandas as pd
import folium


In [27]:
city_coords = {
    "Beirut": (33.8938, 35.5018),
    "Baghdad": (33.3152, 44.3661),
    "Damascus": (33.5138, 36.2765),
    "Jerusalem": (31.7683, 35.2137)
}


In [28]:
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml"
]


In [29]:
def fetch_risk_data(cities, feeds):
    counts = {city: 0 for city in cities}
    for feed_url in feeds:
        feed = feedparser.parse(feed_url)
        for entry in feed.entries:
            title = entry.title.lower()
            for city in cities:
                if city.lower() in title:
                    counts[city] += 1
    return counts


In [30]:
def get_risk_level(count):
    if count > 5:
        return "High"
    elif count > 2:
        return "Medium"
    elif count >= 0:
        return "Low"
    else:
        return "No data"


In [31]:
!pip install feedparser pandas folium


In [32]:
import feedparser
import pandas as pd
import folium
from datetime import datetime

# Define cities and their coordinates
cities = {
    "Gaza": (31.5, 34.47),
    "Beirut": (33.89, 35.50),
    "Baghdad": (33.31, 44.36),
    "Damascus": (33.51, 36.29)
}

# RSS feeds from major open sources
rss_feeds = [
    "http://feeds.bbci.co.uk/news/world/middle_east/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml",
    "https://www.dw.com/en/top-stories/world/s-1429?maca=en-rss-en-all-1573-rdf",
    "https://www.reuters.com/rssFeed/topNews"
]

# Fetch and filter news
def fetch_city_news(city):
    city_news = []
    for feed_url in rss_feeds:
        feed = feedparser.parse(feed_url)
        for entry in feed.entries:
            if city.lower() in entry.title.lower() or city.lower() in entry.get("summary", "").lower():
                city_news.append({
                    "city": city,
                    "title": entry.title,
                    "summary": entry.get("summary", ""),
                    "link": entry.link,
                    "published": entry.get("published", "Unknown")
                })
    return city_news

# Collect news for all cities
all_news = []
for city in cities:
    city_articles = fetch_city_news(city)
    all_news.extend(city_articles)

# Convert to DataFrame
df = pd.DataFrame(all_news)

# Assign basic risk score: more news = higher risk
risk_data = df['city'].value_counts().to_dict()

# Display news table (optional)
print("📰 Latest News from Target Cities:\n")
display(df[['city', 'title', 'published', 'link']])

# Create map with risk circles
risk_map = folium.Map(location=[33.5, 40], zoom_start=5)

for city, (lat, lon) in cities.items():
    count = risk_data.get(city, 0)
    risk_score = min(count * 5, 100)  # Max risk score 100
    folium.CircleMarker(
        location=(lat, lon),
        radius=10,
        color='red' if risk_score > 50 else 'orange' if risk_score > 20 else 'green',
        fill=True,
        fill_opacity=0.7,
        popup=f"{city} — Risk Score: {risk_score} ({count} articles)"
    ).add_to(risk_map)

# Show map
risk_map


📰 Latest News from Target Cities:



,city,title,published,link
0,Gaza,The secret system Hamas uses to pay government...,"Wed, 06 Aug 2025 16:48:57 GMT",https://www.bbc.com/news/articles/c1kz42j92jmo...
1,Gaza,"Twenty killed after trucks overturn in Gaza, H...","Wed, 06 Aug 2025 20:06:46 GMT",https://www.bbc.com/news/articles/cz60dy2908do...
2,Gaza,Netanyahu to propose full reoccupation of Gaza...,"Tue, 05 Aug 2025 15:06:52 GMT",https://www.bbc.com/news/articles/cpqv2qjg5vvo...
3,Gaza,"He is a human skeleton, Gaza hostage's brother...","Mon, 04 Aug 2025 17:05:21 GMT",https://www.bbc.com/news/articles/cewyk4ezeedo...
4,Gaza,Hundreds of Israeli ex-officials appeal to Tru...,"Mon, 04 Aug 2025 13:01:22 GMT",https://www.bbc.com/news/articles/crkznje8nz8o...
5,Gaza,World leaders condemn videos of emaciated Isra...,"Sun, 03 Aug 2025 21:15:58 GMT",https://www.bbc.com/news/articles/crr2dwn7q40o...
6,Gaza,Aid group says worker killed by Israeli milita...,"Sun, 03 Aug 2025 21:43:27 GMT",https://www.bbc.com/news/articles/cx2x5eyl676o...
7,Gaza,Trump envoy visits controversial Gaza aid dist...,"Fri, 01 Aug 2025 18:14:15 GMT",https://www.bbc.com/news/articles/c8x5jnl5eygo...
8,Gaza,France suspends Gaza evacuations after student...,"Fri, 01 Aug 2025 17:15:39 GMT",https://www.bbc.com/news/articles/c5yl7n42325o...
9,Gaza,Gazan boy first to be treated in UK for war in...,"Thu, 31 Jul 2025 18:07:39 GMT",https://www.bbc.com/news/articles/cvgn6979n59o...


In [33]:
risk_data = fetch_risk_data(city_coords.keys(), rss_feeds)
print("Risk data:", risk_data)


Risk data: {'Beirut': 0, 'Baghdad': 0, 'Damascus': 1, 'Jerusalem': 1}


In [34]:
m = folium.Map(location=[33.5, 36.5], zoom_start=6)

for city, coords in city_coords.items():
    count = risk_data.get(city, 0)
    risk = get_risk_level(count)
    color = {"High": "red", "Medium": "orange", "Low": "green", "No data": "gray"}.get(risk, "gray")
    folium.Marker(
        location=coords,
        popup=f"<b>{city}</b><br>Articles: {count}<br>Risk: {risk}",
        tooltip=f"{city}: {risk}",
        icon=folium.Icon(color=color)
    ).add_to(m)

m